<a href="https://colab.research.google.com/github/Farzanafshar/audio-super-res/blob/experiments/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U -q kaggle

In [2]:
! mkdir -p ~/.kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"farzanafsharii","key":"146f53b27bf241c2599c20fe20d5e983"}'}

In [4]:
! cp kaggle.json ~/.kaggle/
! kaggle datasets download -d msambare/fer2013

 73% 44.0M/60.3M [00:00<00:00, 107MB/s]
100% 60.3M/60.3M [00:00<00:00, 126MB/s]


In [5]:
import os 
from glob import glob

WEIGHT_FOLDER='/content/drive/MyDrive/weights'  ## for saving weights
DATA_FOLDER='./data/img_fer2013'

if not os.path.exists(WEIGHT_FOLDER):

  os.makedirs(os.path.join(WEIGHT_FOLDER,"Face_pos"))

from zipfile import ZipFile

with ZipFile('/content/fer2013.zip','r') as zipObj:
  zipObj.extractall('./data/')


In [6]:
from keras.layers.reshaping.flatten import Flatten
from keras.layers.core.lambda_layer import Lambda
import tensorflow as tf
from keras.models import load_model
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

import numpy as np
import matplotlib.pyplot as plt


In [7]:
IMAGE_SIZE=[48,48]+[3]
IMAGE_SIZE

[48, 48, 3]

In [8]:
folders=glob('/content/data/train/*')
folders

['/content/data/train/fear',
 '/content/data/train/happy',
 '/content/data/train/neutral',
 '/content/data/train/disgust',
 '/content/data/train/sad',
 '/content/data/train/angry',
 '/content/data/train/surprise']

In [9]:
IMAGE_SIZE=[224,224]

train_path='/content/data/train'
valid_path='/content/data/test'

vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

for layer in vgg.layers:
  layer.trainable=True

folders=glob('/content/data/train/*')

x= Flatten()(vgg.output)
x= Dense(1000,activation='relu')(x)
prediction=Dense(len(folders),activation='softmax')(x)

model=Model(inputs=vgg.input,outputs=prediction)

model.summary()
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

58889256/58889256 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [10]:
from tensorflow.python import test
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 vertical_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

training_set=train_datagen.flow_from_directory('/content/data/train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

test_set=test_datagen.flow_from_directory('/content/data/test',
                                          target_size=(224,224),
                                          batch_size=32,
                                          class_mode='categorical')



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [11]:
training_set.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [12]:
WEIGHT_FOLDER='/content/drive/MyDrive/weights'

In [13]:
checkpoint_face=ModelCheckpoint(os.path.join(WEIGHT_FOLDER,'face_weight.h5'),save_weights_only=True,verbose=1)

In [ ]:
r=model.fit(training_set,
            validation_data=test_set,
            epochs=50,
            steps_per_epoch=len(training_set),
            validation_steps=len(test_set),
            callbacks=[checkpoint_face])

Epoch 1/50
898/898 [==============================] - ETA: 0s - loss: 1.8598 - accuracy: 0.2485
Epoch 1: saving model to /content/drive/MyDrive/weights/face_weight.h5
898/898 [==============================] - 490s 517ms/step - loss: 1.8598 - accuracy: 0.2485 - val_loss: 1.8151 - val_accuracy: 0.2471
Epoch 2/50
898/898 [==============================] - ETA: 0s - loss: 1.8113 - accuracy: 0.2513
Epoch 2: saving model to /content/drive/MyDrive/weights/face_weight.h5
898/898 [==============================] - 469s 522ms/step - loss: 1.8113 - accuracy: 0.2513 - val_loss: 1.8148 - val_accuracy: 0.2471
Epoch 3/50
898/898 [==============================] - ETA: 0s - loss: 1.8112 - accuracy: 0.2513
Epoch 3: saving model to /content/drive/MyDrive/weights/face_weight.h5
898/898 [==============================] - 472s 525ms/step - loss: 1.8112 - accuracy: 0.2513 - val_loss: 1.8135 - val_accuracy: 0.2471
Epoch 4/50
898/898 [==============================] - ETA: 0s - loss: 1.8112 - accuracy: 0.251